# 功能:掃指定範圍的所有ema均線組合,找出獲利/勝率最大的組合.
* 進場:均線黃金交叉
* 出場:均線死忘交叉 或 收盤價低於sma扺扣價
* 方向:多,空,多空雙向

### 範圍開太大會跑很久
### 開啟google drive是為了把結果存檔到CSV檔.不然colab會自動消失.不想用的請自註解掉.

In [37]:
#戴入google drive
from google.colab import drive
drive.mount('/content/drive') # 此處需要登入google帳號


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
try:
    import vectorbt
except ModuleNotFoundError as e:
    !pip install vectorbt    

try:
    import yfinance
except ModuleNotFoundError as e:
    !pip install yfinance   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt

try:
    import talib
except ModuleNotFoundError as e:
    !pip install talib_binary         


from IPython.display import clear_output
clear_output()

!exit

In [39]:

import vectorbt as vbt
import talib
import pandas as pd
import logging

log_level = logging.INFO

代號名和資料來源是有相關性的.
*   CCXT:應該只支援加密貨幣.ex: BTC/USDT
*   Yahoo_Finance:台股,美股,加密貨幣都支援. ex: 2357.tw, TSLA, BTC-USD
(https://finance.yahoo.com/)

timeframe和資料來源也是有相關性的.
*   CCXT: ['1m', '5m', '15m', '1h', '4h', '1d', '3d']
*   Yahoo_finance: ['1m', '2m', '5m', '15m', '30m', '60', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']



In [40]:
#@title 回測設定
symbol_name = "TSLA" #@param {type:"string"}
data_source = "Yahoo_Finance" #@param ["CCXT","Yahoo_Finance"]

#ccxt timeframe = ['1m', '5m', '15m', '1h', '4h', '1d', '3d']
timeframe_list = ['1h','1d']

date_start = "2021-01-01" #@param {type:"date"}
date_end = "2022-09-21" #@param {type:"date"}

ema_s_start = 8 #@param {type:"integer"}
ema_s_end = 30 #@param {type:"integer"}

ema_m_start = 12 #@param {type:"integer"}
ema_m_end = 60 #@param {type:"integer"}

sma_deduct_start = 7 #@param {type:"integer"}
sma_deduct_end = 25 #@param {type:"integer"}


init_cash = 100 #@param {type:"integer"}

direction = "both" #@param ["longonly","shortonly","both"]
#
multi_processing = False 


ema_s_list = [ema for ema in range(ema_s_start, ema_s_end+1)]
ema_m_list = [ema for ema in range(ema_m_start, ema_m_end+1)]
sma_d_list = [0]+[d for d in range(sma_deduct_start,sma_deduct_end+1)]

In [ ]:
#download data

ohlc_data = dict()
if data_source == "CCXT":
    for i in timeframe_list:
        data = vbt.CCXTData.download(
            symbols=symbol_name,
            start=date_start,
            end=date_end,
            timeframe=i
        )
        ohlc_data.update(
            {i: data}
        )
elif data_source == "Yahoo_Finance":
    for i in timeframe_list:
        data = vbt.YFData.download(
            symbols=symbol_name,
            start=date_start,
            end=date_end,
            interval=i
        )
        ohlc_data.update(
            {i: data}
        )


In [42]:

#define function


def scan_ma(timeframe):

    # create a logger
    logger = logging.getLogger("logger")
    # log all messages, debug and up
    logger.setLevel(log_level)

    results = pd.DataFrame()    
    
    close_price = ohlc_data[timeframe].get("Close")
    
    for i_es in ema_s_list:
        for i_em in ema_m_list:

            ema_sline = talib.EMA(close_price, timeperiod=i_es)
            ema_mline = talib.EMA(close_price, timeperiod=i_em) 

            ema_entry = [False]
            ema_exit = [False]
            sma_d_exit = [False] 

            for i in range(1,ema_sline.shape[0]):
                if ema_sline[i] > ema_mline[i] and ema_sline[i-1] < ema_mline[i-1]:
                    ema_entry.append(True)
                else:
                    ema_entry.append(False)

                if ema_sline[i] < ema_mline[i] and ema_sline[i-1] > ema_mline[i-1]:
                    ema_exit.append(True)
                else:
                    ema_exit.append(False)

                
            for i_sd in sma_d_list:
                
                if i_sd == 0:
                    pf = vbt.Portfolio.from_signals(
                        close=close_price,
                        entries=ema_entry,
                        exits=ema_exit,
                        init_cash=init_cash,
                        direction=direction,
                        
                    )
                else:
                    sma_d_exit=[False]    
                    for i in range(1,close_price.shape[0]):
                        if close_price[i] < close_price[i-i_sd]:
                            sma_d_exit.append(True)
                        else:
                            sma_d_exit.append(False)    

                    pf = vbt.Portfolio.from_signals(
                        close=close_price,
                        entries=ema_entry,
                        exits=sma_d_exit,
                        init_cash=init_cash,
                        direction=direction
                    )   
                temp_series = pd.Series([timeframe,i_es,i_em,i_sd],index=["timeframe","ema_s","ema_m","sma_d"])

                results = pd.concat([results,temp_series.append(pf.stats(silence_warnings=True))],axis=1)
                
                logger.info(f"test{timeframe}x{i_es}x{i_em}x{i_sd}")
                
   
    
    return results



In [ ]:

#main 

import warnings
#warnings.filterwarnings('ignore')

import multiprocessing as mp

cpus = mp.cpu_count()
pool = mp.Pool(processes=cpus)

result_df = pd.DataFrame()

if multi_processing is True:

    result = pool.map(scan_ma,timeframe_list)

    for i in result:
        result_df = pd.concat([result_df,i],axis=1)
    result_df = result_df.T
    result_df.reset_index(drop=True,inplace=True)
else:
    
    for i in timeframe_list:
        result = scan_ma(i)
        result_df = pd.concat([result_df,result],axis=1,ignore_index=True)
    result_df = result_df.T

print(f"Full combination count: {result_df.shape[0]}")

In [44]:
#依獲利排序，取前100, 存到google drive ,檔案格式csv
import os

result_df.sort_values("End Value", ascending=False, inplace=True)
result_df.head(100).to_csv(f"{symbol_name.replace('/','')}_endvaluetable.csv")
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/"):
    result_df.head(100).to_csv(f"/content/drive/MyDrive/Colab Notebooks/{symbol_name.replace('/','')}_endvaluetable.csv")
result_df.head(10)


,timeframe,ema_s,ema_m,sma_d,Start,End,Period,Start Value,End Value,Total Return [%],...,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy
24573,1d,10,15,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,325.542579,225.542579,...,-14.542077,90.0,53.181373,-7.784344,16.448801,-7.784344,43.333333,23.0,25.32448,24.008466
23613,1d,9,16,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,325.542579,225.542579,...,-14.542077,90.0,53.181373,-7.784344,16.448801,-7.784344,43.333333,23.0,25.32448,24.008466
23553,1d,9,13,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,313.81481,213.81481,...,-14.018195,80.0,53.181373,-7.784344,18.18041,-4.45751,48.5,12.5,22.268795,22.783301
24513,1d,10,12,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,313.81481,213.81481,...,-14.018195,80.0,53.181373,-7.784344,18.18041,-4.45751,48.5,12.5,22.268795,22.783301
22593,1d,8,14,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,313.81481,213.81481,...,-14.018195,80.0,53.181373,-7.784344,18.18041,-4.45751,48.5,12.5,22.268795,22.783301
22673,1d,8,18,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,312.6151,212.6151,...,-13.964603,90.0,53.181373,-7.784344,15.902956,-7.784344,43.333333,23.0,23.956209,22.65797
22553,1d,8,12,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,309.785797,209.785797,...,-13.838218,75.0,52.70788,-8.118545,16.055226,-3.162613,42.777778,9.333333,19.830291,18.635335
23533,1d,9,12,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,309.304082,209.304082,...,-13.816699,75.0,52.70788,-8.118545,16.046611,-3.189795,42.888889,9.0,19.67967,18.593398
22573,1d,8,13,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,309.304082,209.304082,...,-13.816699,75.0,52.70788,-8.118545,16.046611,-3.189795,42.888889,9.0,19.67967,18.593398
25493,1d,11,12,19,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,307.662893,207.662893,...,-13.743387,80.0,53.181373,-7.784344,17.914743,-4.45751,48.5,12.5,22.035108,22.140628


In [45]:
#依Win Rate排序，取前100, 存到csv檔

result_df.sort_values("Win Rate [%]", ascending=False, inplace=True)
result_df.head(100).to_csv(f"{symbol_name.replace('/','')}_winratetable.csv")
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/"):
    result_df.head(100).to_csv(f"/content/drive/MyDrive/Colab Notebooks/{symbol_name.replace('/','')}_winratetable.csv")
result_df.head(10)

,timeframe,ema_s,ema_m,sma_d,Start,End,Period,Start Value,End Value,Total Return [%],...,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy
29401,1d,15,12,7,2021-01-04 00:00:00+00:00,2022-09-20 00:00:00+00:00,432,100.0,85.23735,-14.76265,...,-38.028978,100.0,12.287957,9.77698,11.032468,NaN,75.5,NaN,inf,11.633164
19883,1h,28,26,9,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.08673,-25.91327,...,-35.396671,100.0,4.394364,0.4493,2.300972,NaN,671.5,NaN,inf,2.37085
20844,1h,29,25,10,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.661274,-25.338726,...,-29.010834,100.0,2.630433,1.014977,1.822705,NaN,260.0,NaN,inf,1.836054
19884,1h,28,26,10,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.661274,-25.338726,...,-29.010834,100.0,2.630433,1.014977,1.822705,NaN,260.0,NaN,inf,1.836054
20908,1h,29,28,14,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.340445,-25.659555,...,-29.786035,100.0,3.091271,1.004167,2.047719,NaN,264.5,NaN,inf,2.06324
21868,1h,30,27,14,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.340445,-25.659555,...,-29.786035,100.0,3.091271,1.004167,2.047719,NaN,264.5,NaN,inf,2.06324
21803,1h,30,24,9,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.08673,-25.91327,...,-35.396671,100.0,4.394364,0.4493,2.300972,NaN,671.5,NaN,inf,2.37085
20843,1h,29,25,9,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,74.08673,-25.91327,...,-35.396671,100.0,4.394364,0.4493,2.300972,NaN,671.5,NaN,inf,2.37085
17903,1h,26,25,9,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,73.550548,-26.449452,...,-35.140497,100.0,4.394364,0.4493,2.115781,NaN,671.5,NaN,inf,2.172761
20661,1h,29,16,7,2021-01-04 14:30:00+00:00,2022-09-20 19:30:00+00:00,3021,100.0,75.055079,-24.944921,...,-30.354535,100.0,3.777082,1.573115,2.675098,NaN,540.5,NaN,inf,2.704807


# 驗證正確性
## 以TSLA為例:

底下為測試的均線範圍.總共45080組合,跑了5個小時.

![](https://i.imgur.com/lx0rN5T.png)

驗證數據為1d,ema_s:11,ema_m:14,sma_deduct:0.多空雙向

因為tradingview上的策略都沒有sma扺扣,所以只能選扺扣為0的記錄.

對比獲利,已平倉交易,勝率這3項數據,看來是相當接近的結果.

其實這2條線如此接近,回測結果又好,滿出乎意料的.

![](https://i.imgur.com/KldTLwG.png)

tradingview 上搜尋EMA Cross Strategy.初始資金設100.
![](https://i.imgur.com/NcyqJxx.png)

獲利前10,配上這勝率,這真的很罕見.
![](https://i.imgur.com/XFsHwvF.png)


[最後付上測試結果檔案](https://drive.google.com/file/d/1-33IyZeZgvKpUtvRW8svpXinmwTcG7N0/view?usp=sharing)